This mini-post shows a code snippet to get data from [Intrinio](https://intrinio.com) with Python.

Prior to running this code, please set up your environment paths `INTRINIO_API_USERNAME` and `INTRINIO_API_PASSWORD` (i.e. by setting environment variables on Windows or including these variables in `.bashrc` in Linux).

In [12]:
from http.client import HTTPSConnection
from base64 import b64encode
import os
import json

#This sets up the https connection
c = HTTPSConnection("api.intrinio.com")

# Prepare user and password headers.
userAndPass = b64encode(bytearray("{}:{}".format(os.environ['INTRINIO_API_USERNAME'],
                                        os.environ['INTRINIO_API_PASSWORD']), "utf-8")).decode("ascii")
headers = { 'Authorization' : 'Basic %s' %  userAndPass }

# Connect
start_date = '1950-01-03'
end_date = '1950-01-10'
c.request('GET', '/prices?identifier=$SPX&start_date={}&end_date={}'.format(start_date, end_date), headers=headers)

# Read response
res = c.getresponse()
json_data = res.read()

# Convert json to Python dict object.
data = json.loads(json_data)

In [3]:
data

{'api_call_credits': 1,
 'current_page': 1,
 'data': [{'adj_factor': 1,
   'close': 17.030001,
   'date': '1950-01-10',
   'high': 17.030001,
   'low': 17.030001,
   'open': 17.030001,
   'volume': 2160000.0},
  {'adj_factor': 1,
   'close': 17.08,
   'date': '1950-01-09',
   'high': 17.08,
   'low': 17.08,
   'open': 17.08,
   'volume': 2520000.0},
  {'adj_factor': 1,
   'close': 16.98,
   'date': '1950-01-06',
   'high': 16.98,
   'low': 16.98,
   'open': 16.98,
   'volume': 2010000.0},
  {'adj_factor': 1,
   'close': 16.93,
   'date': '1950-01-05',
   'high': 16.93,
   'low': 16.93,
   'open': 16.93,
   'volume': 2550000.0},
  {'adj_factor': 1,
   'close': 16.85,
   'date': '1950-01-04',
   'high': 16.85,
   'low': 16.85,
   'open': 16.85,
   'volume': 1890000.0},
  {'adj_factor': 1,
   'close': 16.66,
   'date': '1950-01-03',
   'high': 16.66,
   'low': 16.66,
   'open': 16.66,
   'volume': 1260000.0}],
 'page_size': 100,
 'result_count': 6,
 'total_pages': 1}

To download while showing progress bar, the following python code may be used.

In [ ]:
from click import progressbar
import requests
import os
from io import BytesIO
import tarfile
import json

ONE_MEGABYTE = 1024 * 1024

start_date = '1950-01-03'
end_date = '1950-01-10'
url = "https://api.intrinio.com/prices?ticker=$SPX&start_date={}&end_date={}".format(start_date, end_date)

resp = requests.get(url, stream=True,
                    auth=(os.environ['INTRINIO_API_USERNAME'], os.environ['INTRINIO_API_PASSWORD']))
resp.raise_for_status()

total_size = int(resp.headers['content-length'])
data_b = BytesIO()
with progressbar(length=total_size,
                 label="Downloading Intrinio prices") as pbar:
    for chunk in resp.iter_content(chunk_size=ONE_MEGABYTE):
        data_b.write(chunk)
        pbar.update(len(chunk))

data_b.seek(0)

output_dir = 'intrinio-data'

data = json.loads(data_b.getvalue())
print(data)